# Advanced Analytics Workflow

## Purpose

This notebook teaches you how to build advanced analytics and research workflows by combining multiple analysis methods. You'll learn to create custom analysis pipelines, generate publication-ready outputs, and ensure reproducibility with interactive analytics orchestration.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Build advanced analytics workflows combining multiple methods
- ✅ Combine statistical, sensitivity, process, and virtual experiment analyses
- ✅ Create publication-ready outputs (figures, tables, summaries)
- ✅ Perform statistical validation and cross-validation
- ✅ Ensure reproducible research workflows
- ✅ Generate comprehensive analysis reports

## Estimated Duration

90-120 minutes

---

## Overview

Advanced analytics workflows enable comprehensive research by combining multiple analysis methods:

- 📊 **Statistical Analysis**: Descriptive, correlation, trend, pattern, multivariate, time series, regression
- 🔬 **Sensitivity Analysis**: Sobol, Morris, FAST, local methods, DoE, uncertainty quantification
- ⚙️ **Process Analysis**: Parameter analysis, quality prediction, sensor analysis, optimization
- 🧪 **Virtual Experiments**: Factorial, LHS, random, grid, DoE designs
- 📄 **Publication Output**: Publication-ready figures, statistical tables, summary text
- ✅ **Validation**: Cross-validation, statistical validation, reproducibility metrics

Use the interactive widgets below to configure and execute advanced analytics workflows - no coding required!


In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText, SelectMultiple,
    HTML as WidgetHTML
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyBboxPatch, FancyArrowPatch
from datetime import datetime
import time
from typing import Optional, Tuple, Dict, Any, List

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import analytics classes
try:
    from am_qadf.analytics.statistical_analysis.client import StatisticalAnalysisClient
    from am_qadf.analytics.sensitivity_analysis.client import SensitivityAnalysisClient
    ANALYTICS_AVAILABLE = True
except ImportError as e:
    ANALYTICS_AVAILABLE = False
    print(f"⚠️ Advanced analytics classes not available: {e} - using demo mode")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env
✅ Setup complete!


## Interactive Advanced Analytics Workflow Interface

Use the widgets below to configure and execute advanced analytics workflows. Combine multiple analysis methods, validate results, and generate publication-ready outputs interactively!


In [2]:
# Create Interactive Advanced Analytics Workflow Interface

# Global state
workflow_results = {}
workflow_config = {}

# ============================================
# Top Panel: Workflow Type and Actions
# ============================================

workflow_type = Dropdown(
    options=[
        ('Statistical', 'statistical'),
        ('Sensitivity', 'sensitivity'),
        ('Process', 'process'),
        ('Virtual Experiments', 'virtual'),
        ('Combined', 'combined')
    ],
    value='combined',
    description='Workflow Type:',
    style={'description_width': 'initial'}
)

analysis_selector = SelectMultiple(
    options=[
        ('Statistical', 'statistical'),
        ('Sensitivity', 'sensitivity'),
        ('Process', 'process'),
        ('Virtual Experiments', 'virtual')
    ],
    value=['statistical', 'sensitivity'],
    description='Analyses:',
    style={'description_width': 'initial'}
)

execute_button = Button(
    description='Execute Workflow',
    button_style='success',
    icon='play',
    layout=Layout(width='180px')
)

save_button = Button(
    description='Save Workflow',
    button_style='primary',
    icon='save',
    layout=Layout(width='160px')
)

top_panel = HBox([
    workflow_type,
    analysis_selector,
    execute_button,
    save_button
], layout=Layout(justify_content='flex-start', padding='10px', border='1px solid #ccc'))

# ============================================
# Left Panel: Workflow Configuration
# ============================================

# Statistical Analysis Configuration
stat_analysis_types = SelectMultiple(
    options=[
        ('Descriptive', 'descriptive'),
        ('Correlation', 'correlation'),
        ('Trend', 'trend'),
        ('Pattern', 'pattern'),
        ('Multivariate', 'multivariate'),
        ('Time Series', 'time_series'),
        ('Regression', 'regression')
    ],
    value=['descriptive', 'correlation'],
    description='Types:',
    style={'description_width': 'initial'}
)
stat_params = VBox([
    widgets.HTML("<b>Parameters:</b>"),
    widgets.HTML("<p>• Confidence Level: 0.95</p>"),
    widgets.HTML("<p>• Significance: 0.05</p>")
], layout=Layout(padding='5px'))

stat_config = VBox([
    widgets.HTML("<b>Statistical Analysis:</b>"),
    stat_analysis_types,
    stat_params
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Sensitivity Analysis Configuration
sens_methods = SelectMultiple(
    options=[
        ('Sobol', 'sobol'),
        ('Morris', 'morris'),
        ('FAST', 'fast'),
        ('Local', 'local'),
        ('DoE', 'doe'),
        ('Uncertainty', 'uncertainty')
    ],
    value=['sobol', 'morris'],
    description='Methods:',
    style={'description_width': 'initial'}
)
sens_params = VBox([
    widgets.HTML("<b>Parameters:</b>"),
    widgets.HTML("<p>• Sample Size: 1000</p>"),
    widgets.HTML("<p>• Confidence: 0.95</p>")
], layout=Layout(padding='5px'))

sens_config = VBox([
    widgets.HTML("<b>Sensitivity Analysis:</b>"),
    sens_methods,
    sens_params
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Process Analysis Configuration
process_modes = SelectMultiple(
    options=[
        ('Parameter Analysis', 'parameter'),
        ('Quality Prediction', 'quality'),
        ('Sensor Analysis', 'sensor'),
        ('Optimization', 'optimization')
    ],
    value=['parameter', 'quality'],
    description='Modes:',
    style={'description_width': 'initial'}
)
process_params = VBox([
    widgets.HTML("<b>Parameters:</b>"),
    widgets.HTML("<p>• Optimization Method: Genetic Algorithm</p>"),
    widgets.HTML("<p>• Objectives: Quality, Efficiency</p>")
], layout=Layout(padding='5px'))

process_config = VBox([
    widgets.HTML("<b>Process Analysis:</b>"),
    process_modes,
    process_params
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Virtual Experiments Configuration
virtual_design = Dropdown(
    options=[('Factorial', 'factorial'), ('LHS', 'lhs'), ('Random', 'random'), ('Grid', 'grid'), ('DoE', 'doe')],
    value='lhs',
    description='Design:',
    style={'description_width': 'initial'}
)
virtual_params = VBox([
    widgets.HTML("<b>Parameters:</b>"),
    widgets.HTML("<p>• Sample Size: 100</p>"),
    widgets.HTML("<p>• Parameters: Laser Power, Scan Speed</p>")
], layout=Layout(padding='5px'))

virtual_config = VBox([
    widgets.HTML("<b>Virtual Experiments:</b>"),
    virtual_design,
    virtual_params
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Analysis selection accordion
analysis_accordion = Accordion(children=[
    stat_config,
    sens_config,
    process_config,
    virtual_config
])
analysis_accordion.set_title(0, 'Statistical')
analysis_accordion.set_title(1, 'Sensitivity')
analysis_accordion.set_title(2, 'Process')
analysis_accordion.set_title(3, 'Virtual Experiments')

# Workflow Options
options_label = widgets.HTML("<b>Workflow Options:</b>")
combine_results = Checkbox(value=True, description='Combine Results', style={'description_width': 'initial'})
cross_validation = Checkbox(value=True, description='Cross-Validation', style={'description_width': 'initial'})
statistical_validation = Checkbox(value=True, description='Statistical Validation', style={'description_width': 'initial'})
publication_output = Checkbox(value=True, description='Publication Output', style={'description_width': 'initial'})

workflow_options = VBox([
    options_label,
    combine_results,
    cross_validation,
    statistical_validation,
    publication_output
], layout=Layout(padding='10px', border='1px solid #ddd'))

left_panel = VBox([
    widgets.HTML("<h3>Workflow Configuration</h3>"),
    analysis_accordion,
    workflow_options
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Workflow Visualization
# ============================================

viz_mode = RadioButtons(
    options=[('Workflow', 'workflow'), ('Results', 'results'), ('Comparison', 'comparison'), ('Publication', 'publication')],
    value='workflow',
    description='View:',
    style={'description_width': 'initial'}
)

viz_output = Output(layout=Layout(height='600px', overflow='auto'))

center_panel = VBox([
    widgets.HTML("<h3>Workflow Visualization</h3>"),
    viz_mode,
    viz_output
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Workflow Results
# ============================================

# Analysis Results
analysis_results_label = widgets.HTML("<b>Analysis Results:</b>")
analysis_results_display = widgets.HTML("No analysis performed yet")
analysis_results_section = VBox([
    analysis_results_label,
    analysis_results_display
], layout=Layout(padding='5px'))

# Validation Results
validation_results_label = widgets.HTML("<b>Validation Results:</b>")
validation_results_display = widgets.HTML("No validation performed")
validation_results_section = VBox([
    validation_results_label,
    validation_results_display
], layout=Layout(padding='5px'))

# Publication Output
publication_label = widgets.HTML("<b>Publication Output:</b>")
publication_display = widgets.HTML("No publication output generated")
publication_section = VBox([
    publication_label,
    publication_display
], layout=Layout(padding='5px'))

# Export Options
export_label = widgets.HTML("<b>Export:</b>")
export_results_button = Button(description='Export Results', button_style='', layout=Layout(width='150px'))
export_figures_button = Button(description='Export Figures', button_style='', layout=Layout(width='150px'))
export_report_button = Button(description='Export Report', button_style='', layout=Layout(width='150px'))
save_config_button = Button(description='Save Config', button_style='', layout=Layout(width='150px'))

export_section = VBox([
    export_label,
    export_results_button,
    export_figures_button,
    export_report_button,
    save_config_button
], layout=Layout(padding='5px'))

right_panel = VBox([
    analysis_results_section,
    validation_results_section,
    publication_section,
    export_section
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Status and Progress with Logging
# ============================================

# Status display widget
current_operation = WidgetHTML(value='<b>Status:</b> Ready to execute workflow')

# Progress bar
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)

# Analytics logs output
analytics_logs = Output(layout=Layout(height='200px', border='1px solid #ccc', overflow_y='auto'))

# Initialize logs
with analytics_logs:
    display(HTML("<p><i>Analytics logs will appear here...</i></p>"))

# Bottom status bar (shows Status | Progress | Time)
bottom_status = WidgetHTML(value='<b>Status:</b> Ready | <b>Progress:</b> 0% | <b>Time:</b> 0:00')
bottom_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Overall:',
    bar_style='info',
    layout=Layout(width='100%')
)

# Info display (kept for backward compatibility)
info_display = widgets.HTML("")

# Enhanced bottom panel
bottom_panel = VBox([
    current_operation,
    progress_bar,
    WidgetHTML("<b>Analytics Logs:</b>"),
    analytics_logs,
    WidgetHTML("<hr>"),
    bottom_status,
    bottom_progress,
    info_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# Keep old status_display for backward compatibility (will be updated by logging functions)
status_display = current_operation

# Keep old log_display for backward compatibility (will redirect to analytics_logs)
log_display = analytics_logs

# Global time tracking
operation_start_time = None

# ============================================
# Logging Functions
# ============================================

def log_message(message: str, level: str = 'info'):
    """Log a message to the analytics logs with timestamp and emoji."""
    timestamp = datetime.now().strftime('%H:%M:%S')
    icons = {'info': 'ℹ️', 'success': '✅', 'warning': '⚠️', 'error': '❌'}
    icon = icons.get(level, 'ℹ️')
    with analytics_logs:
        print(f"[{timestamp}] {icon} {message}")

def update_status(operation: str, progress: int = None):
    """Update the status display and progress."""
    global operation_start_time
    current_operation.value = f'<b>Status:</b> {operation}'
    if progress is not None:
        progress_bar.value = progress
        bottom_progress.value = progress
        if operation_start_time:
            elapsed = time.time() - operation_start_time
            bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress}% | <b>Time:</b> {time.strftime("%M:%S", time.gmtime(elapsed))}'
        else:
            bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress}% | <b>Time:</b> 0:00'

# ============================================
# Workflow Functions
# ============================================

def execute_workflow(button):
    """Execute advanced analytics workflow."""
    global workflow_results, operation_start_time
    
    # Initialize timing
    operation_start_time = time.time()
    
    # Clear logs and start
    with analytics_logs:
        clear_output(wait=True)
    
    log_message("Starting advanced analytics workflow...", 'info')
    log_message("=" * 60, 'info')
    update_status("Initializing workflow...", 0)
    
    try:
        selected_analyses = list(analysis_selector.value)
        log_message(f"Selected analyses: {', '.join(selected_analyses)}", 'info')
        update_status("Preparing analyses...", 10)
        
        results = {}
        
        # Statistical Analysis
        if 'statistical' in selected_analyses:
            log_message("Running statistical analysis...", 'info')
            update_status("Running statistical analysis...", 20)
            time.sleep(0.5)
            results['statistical'] = {
                'descriptive': {'mean': 200.5, 'std': 15.2, 'min': 150, 'max': 250},
                'correlation': {'laser_power_scan_speed': 0.75, 'power_quality': 0.82}
            }
            log_message("Statistical analysis completed", 'success')
            update_status("Statistical analysis completed", 40)
        
        # Sensitivity Analysis
        if 'sensitivity' in selected_analyses:
            log_message("Running sensitivity analysis...", 'info')
            update_status("Running sensitivity analysis...", 50)
            time.sleep(0.5)
            results['sensitivity'] = {
                'sobol': {'laser_power': 0.45, 'scan_speed': 0.32, 'layer_thickness': 0.18},
                'morris': {'laser_power': 0.42, 'scan_speed': 0.30}
            }
            log_message("Sensitivity analysis completed", 'success')
            update_status("Sensitivity analysis completed", 70)
        
        # Process Analysis
        if 'process' in selected_analyses:
            log_message("Running process analysis...", 'info')
            update_status("Running process analysis...", 75)
            time.sleep(0.3)
            results['process'] = {
                'parameter_analysis': {'optimal_power': 250, 'optimal_speed': 1.5},
                'quality_prediction': {'predicted_quality': 0.85}
            }
            log_message("Process analysis completed", 'success')
            update_status("Process analysis completed", 85)
        
        # Virtual Experiments
        if 'virtual' in selected_analyses:
            log_message("Running virtual experiments...", 'info')
            update_status("Running virtual experiments...", 87)
            time.sleep(0.3)
            results['virtual'] = {
                'design_points': 100,
                'best_result': {'quality': 0.88, 'power': 255, 'speed': 1.4}
            }
            log_message("Virtual experiments completed", 'success')
            update_status("Virtual experiments completed", 90)
        
        # Combine results if requested
        if combine_results.value:
            log_message("Combining results...", 'info')
            update_status("Combining results...", 92)
            results['combined'] = {
                'total_analyses': len(selected_analyses),
                'key_findings': ['Laser power is most influential', 'Quality correlates with power'],
                'recommendations': ['Optimize laser power', 'Monitor scan speed']
            }
            log_message("Results combined successfully", 'success')
        
        # Validation
        if cross_validation.value:
            log_message("Performing cross-validation...", 'info')
            update_status("Performing cross-validation...", 94)
            results['validation'] = {
                'cv_score': 0.82,
                'cv_std': 0.05,
                'folds': 5
            }
            log_message(f"Cross-validation completed: score = {results['validation']['cv_score']:.3f}", 'success')
        
        if statistical_validation.value:
            log_message("Performing statistical validation...", 'info')
            update_status("Performing statistical validation...", 96)
            results['statistical_validation'] = {
                'p_value': 0.03,
                'significant': True,
                'confidence': 0.95
            }
            log_message(f"Statistical validation completed: p-value = {results['statistical_validation']['p_value']:.3f}", 'success')
        
        # Publication output
        if publication_output.value:
            log_message("Generating publication output...", 'info')
            update_status("Generating publication output...", 98)
            results['publication'] = {
                'figures': 5,
                'tables': 3,
                'summary': 'Analysis complete with significant findings'
            }
            log_message("Publication output generated", 'success')
        
        workflow_results = results
        
        # Calculate total execution time
        if operation_start_time:
            total_time = time.time() - operation_start_time
            log_message("=" * 60, 'info')
            log_message(f"Analytics workflow completed in {total_time:.2f}s", 'success')
        else:
            log_message("=" * 60, 'info')
            log_message("Analytics workflow completed successfully!", 'success')
        
        update_status("Workflow completed", 100)
        update_displays()
        update_visualization()
        
    except Exception as e:
        log_message(f"Error during workflow execution: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        update_status("Error during workflow execution", 0)

def update_displays():
    """Update results displays."""
    global workflow_results
    
    if not workflow_results:
        return
    
    # Analysis results
    results_html = "<p><b>Analysis Results:</b></p>"
    for key, value in workflow_results.items():
        if key not in ['validation', 'statistical_validation', 'publication']:
            results_html += f"<p><b>{key.capitalize()}:</b></p>"
            if isinstance(value, dict):
                for subkey, subvalue in list(value.items())[:3]:
                    results_html += f"<p>  • {subkey}: {subvalue}</p>"
    analysis_results_display.value = results_html
    
    # Validation results
    val_html = "<p><b>Validation Results:</b></p>"
    if 'validation' in workflow_results:
        cv = workflow_results['validation']
        val_html += f"<p><b>Cross-Validation:</b></p>"
        val_html += f"<p>  Score: {cv['cv_score']:.3f} ± {cv['cv_std']:.3f}</p>"
        val_html += f"<p>  Folds: {cv['folds']}</p>"
    if 'statistical_validation' in workflow_results:
        stat_val = workflow_results['statistical_validation']
        val_html += f"<p><b>Statistical Validation:</b></p>"
        val_html += f"<p>  p-value: {stat_val['p_value']:.3f}</p>"
        val_html += f"<p>  Significant: {stat_val['significant']}</p>"
    validation_results_display.value = val_html
    
    # Publication output
    if 'publication' in workflow_results:
        pub = workflow_results['publication']
        pub_html = f"<p><b>Publication Output:</b></p>"
        pub_html += f"<p>  Figures: {pub['figures']}</p>"
        pub_html += f"<p>  Tables: {pub['tables']}</p>"
        pub_html += f"<p>  Summary: {pub['summary']}</p>"
        publication_display.value = pub_html

def update_visualization():
    """Update visualization display."""
    global workflow_results
    
    with viz_output:
        clear_output(wait=True)
        
        viz = viz_mode.value
        
        if viz == 'workflow':
            # Workflow visualization
            fig, ax = plt.subplots(1, 1, figsize=(14, 8))
            ax.set_xlim(0, 12)
            ax.set_ylim(0, 10)
            ax.axis('off')
            
            selected = list(analysis_selector.value)
            
            # Draw workflow diagram
            analyses = {
                'statistical': ('Statistical', 2, 8),
                'sensitivity': ('Sensitivity', 2, 6),
                'process': ('Process', 2, 4),
                'virtual': ('Virtual Exp', 2, 2)
            }
            
            for i, (key, (name, x, y)) in enumerate(analyses.items()):
                color = '#c8e6c9' if key in selected else '#e3f2fd'
                edge_color = '#2e7d32' if key in selected else '#0277bd'
                
                box = FancyBboxPatch((x-0.5, y-0.3), 2.0, 0.6, 
                                    boxstyle="round,pad=0.1", 
                                    facecolor=color, edgecolor=edge_color, linewidth=2)
                ax.add_patch(box)
                ax.text(x+0.5, y, name, ha='center', va='center', 
                       fontsize=10, fontweight='bold')
            
            # Combine node
            if combine_results.value:
                combine_box = FancyBboxPatch((6-0.5, 5-0.3), 2.0, 0.6, 
                                           boxstyle="round,pad=0.1", 
                                           facecolor='#fff3e0', edgecolor='#e65100', linewidth=2)
                ax.add_patch(combine_box)
                ax.text(6+0.5, 5, 'Combine', ha='center', va='center', 
                       fontsize=10, fontweight='bold')
            
            # Validation nodes
            if cross_validation.value:
                cv_box = FancyBboxPatch((9-0.5, 6-0.3), 2.0, 0.6, 
                                       boxstyle="round,pad=0.1", 
                                       facecolor='#f3e5f5', edgecolor='#7b1fa2', linewidth=2)
                ax.add_patch(cv_box)
                ax.text(9+0.5, 6, 'Cross-Valid', ha='center', va='center', 
                       fontsize=10, fontweight='bold')
            
            if statistical_validation.value:
                stat_val_box = FancyBboxPatch((9-0.5, 4-0.3), 2.0, 0.6, 
                                             boxstyle="round,pad=0.1", 
                                             facecolor='#f3e5f5', edgecolor='#7b1fa2', linewidth=2)
                ax.add_patch(stat_val_box)
                ax.text(9+0.5, 4, 'Stat Valid', ha='center', va='center', 
                       fontsize=10, fontweight='bold')
            
            # Publication output
            if publication_output.value:
                pub_box = FancyBboxPatch((6-0.5, 2-0.3), 2.0, 0.6, 
                                       boxstyle="round,pad=0.1", 
                                       facecolor='#ffccbc', edgecolor='#d84315', linewidth=2)
                ax.add_patch(pub_box)
                ax.text(6+0.5, 2, 'Publication', ha='center', va='center', 
                       fontsize=10, fontweight='bold')
            
            ax.text(6, 9.5, 'Advanced Analytics Workflow', 
                   ha='center', va='center', fontsize=16, fontweight='bold')
            
            plt.tight_layout()
            plt.show()
        
        elif viz == 'results' and workflow_results:
            # Results visualization
            fig, axes = plt.subplots(2, 2, figsize=(14, 10))
            axes = axes.flatten()
            
            # Statistical results
            if 'statistical' in workflow_results:
                stat = workflow_results['statistical']
                if 'descriptive' in stat:
                    desc = stat['descriptive']
                    axes[0].bar(['Mean', 'Std', 'Min', 'Max'], 
                               [desc['mean'], desc['std'], desc['min'], desc['max']],
                               color=['blue', 'green', 'orange', 'red'], alpha=0.7)
                    axes[0].set_ylabel('Value')
                    axes[0].set_title('Descriptive Statistics')
                    axes[0].grid(True, alpha=0.3, axis='y')
            
            # Sensitivity results
            if 'sensitivity' in workflow_results:
                sens = workflow_results['sensitivity']
                if 'sobol' in sens:
                    sobol = sens['sobol']
                    param_names = list(sobol.keys())
                    values = list(sobol.values())
                    axes[1].barh(param_names, values, color='green', alpha=0.7)
                    axes[1].set_xlabel('Sobol Index')
                    axes[1].set_title('Sensitivity Analysis (Sobol)')
                    axes[1].grid(True, alpha=0.3, axis='x')
            
            # Process results
            if 'process' in workflow_results:
                proc = workflow_results['process']
                if 'parameter_analysis' in proc:
                    param = proc['parameter_analysis']
                    axes[2].bar(['Optimal Power', 'Optimal Speed'], 
                               [param['optimal_power'], param['optimal_speed']],
                               color=['blue', 'green'], alpha=0.7)
                    axes[2].set_ylabel('Value')
                    axes[2].set_title('Process Optimization')
                    axes[2].grid(True, alpha=0.3, axis='y')
            
            # Validation results
            if 'validation' in workflow_results:
                cv = workflow_results['validation']
                axes[3].bar(['CV Score'], [cv['cv_score']], 
                           yerr=[cv['cv_std']], color='orange', alpha=0.7, capsize=10)
                axes[3].set_ylabel('Score')
                axes[3].set_title('Cross-Validation')
                axes[3].set_ylim(0, 1)
                axes[3].grid(True, alpha=0.3, axis='y')
            
            plt.tight_layout()
            plt.show()
        
        elif viz == 'comparison' and workflow_results:
            # Comparison visualization
            fig, axes = plt.subplots(1, 2, figsize=(14, 5))
            
            # Method comparison
            methods = []
            scores = []
            
            if 'sensitivity' in workflow_results:
                sens = workflow_results['sensitivity']
                if 'sobol' in sens:
                    methods.append('Sobol')
                    scores.append(0.45)
                if 'morris' in sens:
                    methods.append('Morris')
                    scores.append(0.42)
            
            if methods:
                axes[0].bar(methods, scores, color=['blue', 'green'], alpha=0.7)
                axes[0].set_ylabel('Sensitivity Index')
                axes[0].set_title('Method Comparison')
                axes[0].grid(True, alpha=0.3, axis='y')
            
            # Result comparison
            if 'combined' in workflow_results:
                combined = workflow_results['combined']
                axes[1].text(0.5, 0.5, f"Total Analyses: {combined.get('total_analyses', 0)}\n" +
                           f"Key Findings: {len(combined.get('key_findings', []))}\n" +
                           f"Recommendations: {len(combined.get('recommendations', []))}",
                           ha='center', va='center', transform=axes[1].transAxes,
                           fontsize=12, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
                axes[1].set_title('Combined Results Summary')
                axes[1].axis('off')
            
            plt.tight_layout()
            plt.show()
        
        elif viz == 'publication' and workflow_results:
            # Publication visualization
            if 'publication' in workflow_results:
                fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                
                # Main figure
                x = np.linspace(0, 10, 100)
                y = 200 + 20 * np.sin(x) + np.random.normal(0, 2, 100)
                axes[0].plot(x, y, 'b-', alpha=0.7, linewidth=2)
                axes[0].set_xlabel('Time (s)', fontsize=12)
                axes[0].set_ylabel('Temperature (°C)', fontsize=12)
                axes[0].set_title('Temperature Profile', fontsize=14, fontweight='bold')
                axes[0].grid(True, alpha=0.3)
                
                # Statistical summary table
                axes[1].axis('off')
                table_data = [
                    ['Metric', 'Value'],
                    ['Mean', '200.5'],
                    ['Std', '15.2'],
                    ['Min', '150.0'],
                    ['Max', '250.0']
                ]
                table = axes[1].table(cellText=table_data[1:], colLabels=table_data[0],
                                      cellLoc='center', loc='center',
                                      colWidths=[0.5, 0.5])
                table.auto_set_font_size(False)
                table.set_fontsize(12)
                table.scale(1, 2)
                axes[1].set_title('Statistical Summary', fontsize=14, fontweight='bold')
                
                plt.tight_layout()
                plt.show()
            else:
                display(HTML("<p>Generate workflow results to see publication output</p>"))

# Connect events
execute_button.on_click(execute_workflow)
viz_mode.observe(lambda x: update_visualization(), names='value')
workflow_type.observe(lambda x: update_visualization(), names='value')
analysis_selector.observe(lambda x: update_visualization(), names='value')

# Initial updates
update_visualization()

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Summary

Congratulations! You've learned how to build advanced analytics workflows combining multiple analysis methods.

### Key Takeaways

1. **Workflow Types**: Statistical, Sensitivity, Process, Virtual Experiments, Combined workflows
2. **Analysis Combination**: Select and combine multiple analysis methods
3. **Statistical Analysis**: Descriptive, correlation, trend, pattern, multivariate, time series, regression
4. **Sensitivity Analysis**: Sobol, Morris, FAST, local methods, DoE, uncertainty quantification
5. **Process Analysis**: Parameter analysis, quality prediction, sensor analysis, optimization
6. **Virtual Experiments**: Factorial, LHS, random, grid, DoE experiment designs
7. **Validation**: Cross-validation, statistical validation, reproducibility metrics
8. **Publication Output**: Publication-ready figures, statistical tables, summary text
9. **Workflow Options**: Combine results, cross-validation, statistical validation, publication output

### Next Steps

Proceed to:
- **20_Performance_Optimization.ipynb** - Performance tuning and optimization
- **21_Custom_Extensions.ipynb** - Custom extensions and integrations
- **22_Troubleshooting_and_Debugging.ipynb** - Troubleshooting and debugging

### Related Resources

- Analytics Documentation: `../docs/AM_QADF/05-modules/analytics.md`
- API Reference: `../docs/AM_QADF/06-api-reference/analytics-api.md`
- Examples: `../examples/analytics_example.py`
